### Criação de Pipeline de Extração, Limpeza, Transformação e Enriquecimento de Dados

#### Etapa 1 

Vamos criar um banco de dados totalmente novo utilizando SQLite. Primeiramente, vamos criar uma tabela denominada "produto". Em seguida, abriremos um arquivo CSV e importaremos os dados desse arquivo para preencher nossa tabela.

In [1]:
# Imports
import csv
import sqlite3

# Cria um novo banco de dados
conn = sqlite3.connect('dsadb.db')

# Cria uma tabela para armazenar os dados de produção de alimentos
conn.execute('''CREATE TABLE producao (
                produto TEXT,
                quantidade INTEGER,
                preco_medio REAL,
                receita_total REAL
            )''')

# Grava e fecha a conexão
conn.commit()
conn.close()

# Abre o arquivo CSV com os dados de produção de alimentos
with open('producao_alimentos.csv', 'r') as file:
    
    # Cria um leitor de CSV para ler o arquivo
    reader = csv.reader(file)

    # Pula a primeira linha, que contém os cabeçalhos das colunas
    next(reader)

    # Conecta ao banco de dados
    conn = sqlite3.connect('dsadb.db')

    # Insere cada linha do arquivo na tabela do banco de dados
    for row in reader:
        conn.execute('INSERT INTO producao (produto, quantidade, preco_medio, receita_total) VALUES (?, ?, ?, ?)', row)

    conn.commit()
    conn.close()

print("Job Concluído com Sucesso!")

Job Concluído com Sucesso!


[Imagem do Banco de dados](C:\Python - DSA\demo4/1.png)

#### Etapa 2

Após revisar a tabela recém-criada e observar que a coluna "receita" foi carregada com uma formatação inadequada, procederemos à abertura do arquivo CSV intitulado "produtos alimentos". Nesse ponto, removeremos a tabela existente e substituiremos por uma nova, permitindo-nos realizar as devidas correções em conformidade com as diretrizes estabelecidas pelas regras de negócio.

#### Regra de negócio 1 : Carregar somente registros com quantidade produzida superior a 10.

In [2]:
# Abre o arquivo CSV com os dados da produção de alimentos
with open('producao_alimentos.csv', 'r') as file:
    
    # Cria um leitor de CSV para ler o arquivo
    reader = csv.reader(file)

    # Pula a primeira linha, que contém os cabeçalhos das colunas
    next(reader)

    # Conecta ao banco de dados
    conn = sqlite3.connect('dsadb.db')

    # Deleta a tabela existente, se houver
    conn.execute('DROP TABLE IF EXISTS producao')

    # Cria uma nova tabela para armazenar os dados de produção de alimentos
    conn.execute('''CREATE TABLE producao (
                    produto TEXT,
                    quantidade INTEGER,
                    preco_medio REAL,
                    receita_total REAL
                )''')

    # Insere cada linha do arquivo com quantidade maior que 10 na tabela do banco de dados
    for row in reader:
        if int(row[1]) > 10:
            conn.execute('INSERT INTO producao (produto, quantidade, preco_medio, receita_total) VALUES (?, ?, ?, ?)', row)

    conn.commit()
    conn.close()

print("Job Concluído com Sucesso!")

Job Concluído com Sucesso!


#### Etapa 3

Vamos desenvolver uma função que elimine os pontos da coluna "receita_total", uma vez que essa coluna não está apresentando com precisão os valores das receitas. Adicionalmente, iremos implementar a primeira regra de negócio e incorporar a segunda regra de negócio, que inclui a conversão do valor para um número inteiro.

#### Regra de negócio 2 : Remover o caracter "ponto" na última coluna do arquivo para evitar que o número seja truncado 

In [3]:
# Função para remover o ponto nos dados da última coluna
def remove_ponto(valor):
    return int(valor.replace('.', ''))

# Abre o arquivo CSV com os dados da produção de alimentos
with open('producao_alimentos.csv', 'r') as file:
    
    # Cria um leitor de CSV para ler o arquivo
    reader = csv.reader(file)

    # Pula a primeira linha, que contém os cabeçalhos das colunas
    next(reader)

    # Conecta ao banco de dados
    conn = sqlite3.connect('dsadb.db')

    # Deleta a tabela existente, se houver
    conn.execute('DROP TABLE IF EXISTS producao')

    # Cria uma nova tabela para armazenar os dados de produção de alimentos
    conn.execute('''CREATE TABLE producao (
                    produto TEXT,
                    quantidade INTEGER,
                    preco_medio REAL,
                    receita_total INTEGER
                )''')

    # Insere cada linha do arquivo com quantidade maior que 10 na tabela do banco de dados
    for row in reader:
        if int(row[1]) > 10:
            
            # Remove o ponto do valor da última coluna e converte para inteiro
            row[3] = remove_ponto(row[3])

            # Insere o registro no banco de dados
            conn.execute('INSERT INTO producao (produto, quantidade, preco_medio, receita_total) VALUES (?, ?, ?, ?)', row)

    conn.commit()
    conn.close()

print("Job Concluído com Sucesso!")


Job Concluído com Sucesso!


#### Etapa 4

Além disso, vamos incorporar a terceira regra de negócio, que envolve a inclusão de uma nova coluna na tabela, denominada "margem de lucro". Essa adição será feita enquanto mantemos as regras anteriores intactas, garantindo assim a integridade das modificações realizadas.

#### Regra de negócio 3 : Enriquecer os dados adicionando no destino uma coluna com a margem de lucro de cada produto

In [4]:
# Função para remover o ponto da última coluna
def remove_ponto(valor):
    return int(valor.replace('.', ''))

# Abre o arquivo CSV com os dados da produção de alimentos
with open('producao_alimentos.csv', 'r') as file:
    
    # Cria um leitor de CSV para ler o arquivo
    reader = csv.reader(file)

    # Pula a primeira linha, que contém os cabeçalhos das colunas
    next(reader)

    # Conecta ao banco de dados
    conn = sqlite3.connect('dsadb.db')

    # Deleta a tabela existente, se houver
    conn.execute('DROP TABLE IF EXISTS producao')

    # Cria uma nova tabela para armazenar os dados de produção de alimentos com a nova coluna 'margem_lucro'
    conn.execute('''CREATE TABLE producao (
                    produto TEXT,
                    quantidade INTEGER,
                    preco_medio REAL,
                    receita_total INTEGER,
                    margem_lucro REAL
                )''')

    # Insere cada linha do arquivo com quantidade maior que 10 na tabela do banco de dados
    for row in reader:
        if int(row[1]) > 10:
            
            # Remove o ponto do valor da última coluna e converte para inteiro
            row[3] = remove_ponto(row[3])

            # Calcula a margem de lucro bruta com base no valor médio de venda e na receita total
            margem_lucro = (row[3] / float(row[1])) - float(row[2])

            # Insere a linha com a nova coluna 'margem_lucro' na tabela do banco de dados
            conn.execute('INSERT INTO producao (produto, quantidade, preco_medio, receita_total, margem_lucro) VALUES (?, ?, ?, ?, ?)', (row[0], row[1], row[2], row[3], margem_lucro))

    conn.commit()
    conn.close()

print("Job Concluído com Sucesso!")


Job Concluído com Sucesso!


#### Etapa 5

Vamos realizar a padronização e formatação do resultado da coluna "margem_lucro" da "receita_total", adotando o mesmo formato da variável "margem_lucro" e aplicando o arredondamento para duas casas decimais por meio do uso da função "round".

In [5]:
# Função para remover o ponto da última coluna
def remove_ponto(valor):
    return int(round(float(valor.replace('.', '')), 0))

# Abre o arquivo CSV com os dados da produção de alimentos
with open('producao_alimentos.csv', 'r') as file:
    
    # Cria um leitor de CSV para ler o arquivo
    reader = csv.reader(file)

    # Pula a primeira linha, que contém os cabeçalhos das colunas
    next(reader)

    # Conecta ao banco de dados
    conn = sqlite3.connect('dsadb.db')

    # Deleta a tabela existente, se houver
    conn.execute('DROP TABLE IF EXISTS producao')

    # Cria uma nova tabela para armazenar os dados de produção de alimentos com a nova coluna 'margem_lucro'
    conn.execute('''CREATE TABLE producao (
                    produto TEXT,
                    quantidade INTEGER,
                    preco_medio REAL,
                    receita_total INTEGER,
                    margem_lucro REAL
                )''')

    # Insere cada linha do arquivo com quantidade maior que 10 na tabela do banco de dados
    for row in reader:
        if int(row[1]) > 10:
            
            # Remove o ponto do valor da última coluna e converte para inteiro
            row[3] = remove_ponto(row[3])

            # Calcula a margem de lucro bruta com base no valor médio de venda e na receita total e arredonda para duas casas decimais
            margem_lucro = round((row[3] / float(row[1])) - float(row[2]), 2)

            # Insere a linha com a nova coluna 'margem_lucro' na tabela do banco de dados
            conn.execute('INSERT INTO producao (produto, quantidade, preco_medio, receita_total, margem_lucro) VALUES (?, ?, ?, ?, ?)', (row[0], row[1], row[2], row[3], margem_lucro))

    conn.commit()
    conn.close()

print("Job Concluído com Sucesso!")

Job Concluído com Sucesso!
